# Import Data Douane To ElasticSearch

### Import des Bibliothèques nécessaires

In [1]:
import pandas as pd
import numpy as np
import os
import json
import time
import elasticsearch
import datetime
from elasticsearch import Elasticsearch
import elasticsearch_dsl
from datetime import datetime
from elasticsearch_dsl import Document, Date, Nested, Boolean, analyzer, InnerDoc, Completion, Keyword, Text, Integer, Keyword, Text
from elasticsearch_dsl.connections import connections
from elasticsearch import helpers

### Création de la variable my_dir (à modifier par  l'utilisateur)

In [3]:
my_dir = r'C:/Users/aaras/Desktop/Data_Douane'
os.chdir(my_dir)

#### Organisation du dossier ' Data_Douane':
* Data_Douane
    * National-2010-export
      *  Libelle_NC8_2010.txt
      *  Libelle_CPF6.txt
      *  Libelle_A129.txt
      *  National_2010_Export.txt
      *  Libelle_PAYS.txt
    * National-2010-import
      *  Libelle_NC8_2010.txt
      *  Libelle_CPF6.txt
      *  Libelle_A129.txt
      *  National_2010_Import.txt
      *  Libelle_PAYS.txt
    * [...]
    * National-2019-export
      *  Libelle_NC8_2010.txt
      *  Libelle_CPF6.txt
      *  Libelle_A129.txt
      *  National_2010_Export.txt
      *  Libelle_PAYS.txt
    * National-2019-import
      *  Libelle_NC8_2019.txt
      *  Libelle_CPF6.txt
      *  Libelle_A129.txt
      *  National_2019_Import.txt
      *  Libelle_PAYS.txt
      

### Création du DataFrame df.import_export

#### Explication des champs
* Flux : le flux correspond au mouvement de la marchandise. Ce champ est valorisé à I pour
Importation et à E pour Exportation
* Mois : c'est le mois durant lequel a eu lieu le mouvement de marchandise.
* Année: c'est l'année durant laquelle a eu lieu le mouvement de marchandise.
* Code CPF6 : C'est le niveau fin de la nomenclature Classification de produits française (CPF).
La CPF est la nomenclature statistique, nationale et centrale de produits. C'est une nomenclature de
produit et de service. Elle permet aux données du commerce extérieur d'être intégrées avec d'autres
statistiques économiques telles que la production industrielle ou la consommation dans des analyses
économiques tant globales que régionales.
* Code A129 : La nomenclature A129 correspond à une nomenclature de regroupement des niveaux
de la CPF pour répondre aux besoins de la production de données de synthèse pour l'analyse
économique et la diffusion.
* Code NC8 : C'est le code de la nomenclature combinée à 8 chiffres (NC8), utilisée pour les
obligations déclaratives des opérateurs auprès de la douane, permet une connaissance détaillée du
commerce extérieur de la France : elle compte en effet un peu moins de 10 000 rubriques.
* Code Pays : C'est un code à deux caractères alphabétiques attribué à chaque pays par la
Commission des communautés européennes.
     * À l'importation, les marchandises sont relevées au compte du pays d'origine.
     * À l'exportation, les envois sont imputés au compte de la destination finale déclarée.
* Valeur : La valeur des échanges est exprimée en euros.
* Masse : Les masses sont exprimées en kilogrammes.
* USUP : Unité supplémentaire.

##### Remarques diverses sur les fichiers utilisés :
* Vérifier que les fichier .txt sont bien encodés en utf-8. Si ce n'est pas le cas vous pouvez ouvrir le fichier dans notepad++ pour le convertir.
* Les headers des fichiers 'Libelle_NC8_xxx.txt' ont été supprimés à la main car il n'apparaissaient pas sur tous les fichiers
* Le code pays de la Namibie étant 'NA', ce code est par défaut détecté par pd.read_csv() comme un NaN. Il faut ajouter une liste de na_values et mettre keep_defaut_na = false la fonction pd.read_csv()
* Ajout de 3 lignes manquantes dans le fichier Libelle_NC8_2018.txt
    *  99500000;Marchandises de faible valeurs dans les échanges intra-communautaires;0;;1993;2018;;
    *  99988000;Composants d'ensembles industriels non classés ailleurs;0;;1993;2018;;
    *  99992000;Colis postaux non classés ailleurs;0;;1993;2018;;
* Ajout de 3 lignes manquantes dans le fichier Libelle_NC8_2019.txt
    *  99050000;Biens personnels appartenant à des personnes physiques qui transfèrent leur résidence normale;0;;2011;2019
    *  99190000;Autres biens appartenant à des personnes physiques qui transfèrent leur résidence normale;0;;2011;2019
    *  99500000;Marchandises de faible valeurs dans les échanges intra-communautaires;0;;1993;2019
    *  99992000;Colis postaux non classés ailleurs;0;;1993;2019

In [4]:
def creation_df_import_export(liste_annee):
    
    '''création d'/un dataframe contenant les données d'/import/export des douanes pour la liste d'années spécifiées en entrée'''
    
    df_columns = ['Code_Flux', 'Libelle_Flux', 'Mois', 'Annee', 'Date_DateTime', 'Code_CPF6', 'Libelle_CPF6', 'Code_A129', 'Libelle_A129', 'Code_NC8', 'Libelle_NC8', 'Code_Pays', 'Libelle_Pays', 'Valeur', 'Masse', 'USUP']
    df = pd.DataFrame(columns=df_columns)
    
    for annee in liste_annee:
        df_import_export = creation_df_import_export_annee(annee)
        df = pd.concat([df,df_import_export])
        print('Année {} ajoutée, {} import/export ajoutés depuis {}'.format(annee, df.shape[0], liste_annee[0]))

    df = df.reset_index(drop = True)
    return df

In [5]:
def creation_df_import_export_annee(annee):
    
    '''Création d'un dataframe pour l'année courante de la forme ['Code_Flux', 'Libelle_Flux', 'Mois', 'Annee', 'Date_DateTime', 'Code_CPF6', 'Libelle_CPF6', 'Code_A129', 'Libelle_A129', 'Code_NC8', 'Libelle_NC8', 'Code_Pays', 'Libelle_Pays', 'Valeur', 'Masse', 'USUP']'''
    
    df_import, df_export, df_nc8, df_pays, df_cpf6, df_a129 = csv_to_df(annee) #on charge les 6 fichiers (import, export, et table de conversion nc8, a129, cpf6, pays)
    df = pd.concat([df_import,df_export]) # on fusionne les imports et les exports
    df = pd.merge(df, df_nc8, how='left') # on ajoute le libelle nc8
    df = pd.merge(df, df_nc8, how='left') # on ajoute le libelle nc8    
    df = pd.merge(df, df_a129, how='left') # on ajoute le libelle a129
    df = pd.merge(df, df_cpf6, how='left') # on ajoute le libelle cpf6
    df = pd.merge(df, df_pays, how='left') # on ajoute le libelle pays
    df = df_add_date_datetime(df) # on ajopute une colonne date au format datetime
    df = df_add_libelle_flux(df) # on ajoute import/export en toute lettre
    new_columns_order = ['Code_Flux', 'Libelle_Flux', 'Mois', 'Annee', 'Date_DateTime', 'Code_CPF6', 'Libelle_CPF6', 'Code_A129', 'Libelle_A129', 'Code_NC8', 'Libelle_NC8', 'Code_Pays', 'Libelle_Pays', 'Valeur', 'Masse', 'USUP']
    df = df.reindex(columns = new_columns_order)
    return df

In [6]:
def csv_to_df(annee):
    
    '''création d'un dataframe contenant les données d'import/export des douanes pour l'année spécifiée en entrée'''
    
    import_dir = os.path.join(my_dir,'National-'+str(annee)+'-import')
    export_dir = os.path.join(my_dir,'National-'+str(annee)+'-export')
    na_values_liste = ['#N/A', '#N/A', 'N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN',' N/A',' NULL', 'NaN', 'n/a','nan', 'null']
    columns_name = ['Code_Flux', 'Mois', 'Annee', 'Code_CPF6','Code_A129','Code_NC8','Code_Pays','Valeur', 'Masse', 'USUP']
    dtype_dico = {'Code_Flux': object , 'Mois': int , 'Annee': int, 'Code_CPF6': object, 'Code_A129': object, 'Code_NC8':object,'Code_Pays':object, 'Valeur':int, 'Masse':int, 'USUP':int}
    os.chdir(import_dir)
    df_import = pd.read_csv('National_'+str(annee)+'_Import.txt', sep=";", header=None, encoding = 'utf-8', names=columns_name, dtype=dtype_dico, keep_default_na=False, na_values = na_values_liste)
    os.chdir(export_dir)
    df_export = pd.read_csv('National_'+str(annee)+'_Export.txt', sep=";", header=None, encoding = 'utf-8', names=columns_name, dtype=dtype_dico, keep_default_na=False, na_values = na_values_liste)
    df_nc8 = pd.read_csv('Libelle_NC8_'+str(annee)+'.txt', sep=";", header=None, names = ['Code_NC8','Libelle_NC8'], encoding = 'utf-8',  usecols = [0,1], dtype={'Code_NC8':object,'Libelle_NC8':object} )
    df_a129 = pd.read_csv('Libelle_A129.txt', sep=";", header=None, names = ['Code_A129','Libelle_A129','c','d'], usecols = ['Code_A129','Libelle_A129'], dtype={'Code_A129':object,'Libelle_A129':object})
    df_cpf6 = pd.read_csv('Libelle_CPF6.txt', sep=";", header=None, names = ['Code_CPF6', 'Libelle_CPF6','c','d'], usecols = ['Code_CPF6', 'Libelle_CPF6'],dtype={'Code_CPF6':object,'Libelle_CPF6':object})
    df_pays = pd.read_csv('Libelle_PAYS.txt', sep=";", header=None, names = ['Code_Pays', 'Libelle_Pays', 'c','d'],usecols=['Code_Pays', 'Libelle_Pays'], dtype={'Code_Pays':object,'Libelle_Pays':object},keep_default_na=False, na_values = na_values_liste)
    return df_import, df_export, df_nc8, df_pays, df_cpf6, df_a129

In [7]:
def df_add_date_datetime(df):
    
    '''A partir des colonnes Annee et Mois du dataframe, création d'une nouvelle colonne Date au format datetime'''
    
    df = df.assign(Date_DateTime = pd.to_datetime({'year':df.Annee,'month':df.Mois,'day':1}))
    return df

In [8]:
def df_add_libelle_flux(df):
    
    '''Création d'une nouvelle colonne spécifiant en toute lettre s'il s'agit d'un import ou d'un export'''
    
    df_flux = pd.DataFrame([['I','Import'],['E','Export']],columns=['Code_Flux', 'Libelle_Flux'])
    df = pd.merge(df, df_flux,how='left')
    return df

In [ ]:
liste_annee = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
df = creation_df_import_export(liste_annee)

Année 2010 ajoutée, 3728478 import/export ajoutés depuis 2010
Année 2011 ajoutée, 7503145 import/export ajoutés depuis 2010
Année 2012 ajoutée, 11392481 import/export ajoutés depuis 2010
Année 2013 ajoutée, 15357223 import/export ajoutés depuis 2010
Année 2014 ajoutée, 19402082 import/export ajoutés depuis 2010
Année 2015 ajoutée, 23524344 import/export ajoutés depuis 2010
Année 2016 ajoutée, 27723531 import/export ajoutés depuis 2010
Année 2017 ajoutée, 31990853 import/export ajoutés depuis 2010
Année 2018 ajoutée, 36339121 import/export ajoutés depuis 2010


## Export vers ElasticSearch


In [ ]:
df_export = df.head(10000)

### Méthode 1:

In [ ]:
INDEX="douane_2"
TYPE= "record"

def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record))

from elasticsearch import Elasticsearch
e = Elasticsearch() # no args, connect to localhost:9200
if not e.indices.exists(INDEX):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%INDEX)

r = e.bulk(rec_to_actions(df)) # return a dict

print(not r["errors"])

### Méthode 2

In [ ]:
index_name="douane"
if not e.indices.exists(index_name):
    raise RuntimeError('index does not exists, use `curl -X PUT "localhost:9200/%s"` and try again'%index_name)


In [ ]:
df['Date_DateTime'] = df['Date_DateTime'].astype(str)

In [ ]:
connections.create_connection(hosts=['localhost'])

class Import_Export_Douane_2(Document):
    flux = Keyword()
    mois = Integer() 
    annee = Integer()
    code_CPF6 = Keyword()
    libelle_CPF6 = Text()
    code_A129 = Keyword()
    libelle_A129 = Text()
    code_NC8 = Keyword()
    libelle_NC8 = Text()
    code_pays = Keyword()
    libelle_pays = Keyword()
    valeur = Integer()
    masse = Integer()
    usup = Integer()
    
    class Index:
        name = 'import_export_douane'

    def save(self, ** kwargs):
        return super(Import_Export_Douane_2, self).save(** kwargs)

    def is_published(self):
        return datetime.now() >= self.published_from

In [ ]:
def import_df(df):
    lis = []
    for i in range (df.shape[0]):
        info_export = df.loc[i].tolist() #['Flux', 'Mois', 'Annee', 'Code_CPF6','Code_A129','Code_NC8','Code_Pays','Valeur', 'Masse', 'USUP'] 
         # create and save and article
        doc = Import_Export_Douane_2(meta={'id': i}, tags=['test'])
        doc.code_flux = info_export[0]
        doc.libelle_flux = info_export[1]
        doc.mois =  info_export[2]
        doc.annee = info_export[3]
        doc.date_datetime = info_export[4]
        doc.code_CPF6 = info_export[5]
        doc.libelle_CPF6 = info_export[6]
        doc.code_A129 = info_export[7]
        doc.libelle_A129 = info_export[8]
        doc.code_NC8 = info_export[9]
        doc.libelle_NC8 = info_export[10]
        doc.code_pays = info_export[11]
        doc.libelle_pays = info_export[12]
        doc.valeur = info_export[13]
        doc.masse = info_export[14]
        doc.usup = info_export[15]
        doc.published_from = datetime.now()
        doc.save(doc_type="test")
        print('Export n°{} importé'.format(i))

    

### Méthode 3 (fastest) 

**Rq**: Méthode plus rapide car on iter en utilisant la methode pandas.dataframe.iterrow()  
__Piste d'amélioration__: utiliser pandas.dataframe.itertuples() (plus rapide)

In [13]:
def doc_generator(df):
    
    df_iter = df.iterrows()
    _id = 0
    for index, document in df_iter:
        _id+=1
        yield {
                "_index": 'douane',
                "_type": "_doc",
                "_id" : f"{str(document['Annee'])+str(document['Mois'])+str(document['Code_Flux'])+str(_id)}",
                "_source": document.to_dict(),
            }
        if _id%1000==0:
            print('Add n°'+str(_id))

In [22]:
es_client = connections.create_connection(hosts=['localhost'],http_auth=("elastic:changeme"),http_compress=True)
tic = time.time()
helpers.bulk(es_client, doc_generator(df))
tac = time.time()
print("L'import a pris "+ str(round(((tac-tic)/60),1))+' min')

Add n°1000
Add n°2000
Add n°3000
Add n°4000
Add n°5000
Add n°6000
Add n°7000
Add n°8000
Add n°9000
Add n°10000
Add n°11000
Add n°12000
Add n°13000
Add n°14000
Add n°15000
Add n°16000
Add n°17000
Add n°18000
Add n°19000
Add n°20000
Add n°21000
Add n°22000
Add n°23000
Add n°24000
Add n°25000
Add n°26000
Add n°27000
Add n°28000
Add n°29000
Add n°30000
Add n°31000
Add n°32000
Add n°33000
Add n°34000
Add n°35000
Add n°36000
Add n°37000
Add n°38000
Add n°39000
Add n°40000
Add n°41000
Add n°42000
Add n°43000
Add n°44000
Add n°45000
Add n°46000
Add n°47000
Add n°48000
Add n°49000
Add n°50000
Add n°51000
Add n°52000
Add n°53000
Add n°54000
Add n°55000
Add n°56000
Add n°57000
Add n°58000
Add n°59000
Add n°60000
Add n°61000
Add n°62000
Add n°63000
Add n°64000
Add n°65000
Add n°66000
Add n°67000
Add n°68000
Add n°69000
Add n°70000
Add n°71000
Add n°72000
Add n°73000
Add n°74000
Add n°75000
Add n°76000
Add n°77000
Add n°78000
Add n°79000
Add n°80000
Add n°81000
Add n°82000
Add n°83000
Add n°84000
A

Add n°640000
Add n°641000
Add n°642000
Add n°643000
Add n°644000
Add n°645000
Add n°646000
Add n°647000
Add n°648000
Add n°649000
Add n°650000
Add n°651000
Add n°652000
Add n°653000
Add n°654000
Add n°655000
Add n°656000
Add n°657000
Add n°658000
Add n°659000
Add n°660000
Add n°661000
Add n°662000
Add n°663000
Add n°664000
Add n°665000
Add n°666000
Add n°667000
Add n°668000
Add n°669000
Add n°670000
Add n°671000
Add n°672000
Add n°673000
Add n°674000
Add n°675000
Add n°676000
Add n°677000
Add n°678000
Add n°679000
Add n°680000
Add n°681000
Add n°682000
Add n°683000
Add n°684000
Add n°685000
Add n°686000
Add n°687000
Add n°688000
Add n°689000
Add n°690000
Add n°691000
Add n°692000
Add n°693000
Add n°694000
Add n°695000
Add n°696000
Add n°697000
Add n°698000
Add n°699000
Add n°700000
Add n°701000
Add n°702000
Add n°703000
Add n°704000
Add n°705000
Add n°706000
Add n°707000
Add n°708000
Add n°709000
Add n°710000
Add n°711000
Add n°712000
Add n°713000
Add n°714000
Add n°715000
Add n°716000

Add n°1251000
Add n°1252000
Add n°1253000
Add n°1254000
Add n°1255000
Add n°1256000
Add n°1257000
Add n°1258000
Add n°1259000
Add n°1260000
Add n°1261000
Add n°1262000
Add n°1263000
Add n°1264000
Add n°1265000
Add n°1266000
Add n°1267000
Add n°1268000
Add n°1269000
Add n°1270000
Add n°1271000
Add n°1272000
Add n°1273000
Add n°1274000
Add n°1275000
Add n°1276000
Add n°1277000
Add n°1278000
Add n°1279000
Add n°1280000
Add n°1281000
Add n°1282000
Add n°1283000
Add n°1284000
Add n°1285000
Add n°1286000
Add n°1287000
Add n°1288000
Add n°1289000
Add n°1290000
Add n°1291000
Add n°1292000
Add n°1293000
Add n°1294000
Add n°1295000
Add n°1296000
Add n°1297000
Add n°1298000
Add n°1299000
Add n°1300000
Add n°1301000
Add n°1302000
Add n°1303000
Add n°1304000
Add n°1305000
Add n°1306000
Add n°1307000
Add n°1308000
Add n°1309000
Add n°1310000
Add n°1311000
Add n°1312000
Add n°1313000
Add n°1314000
Add n°1315000
Add n°1316000
Add n°1317000
Add n°1318000
Add n°1319000
Add n°1320000
Add n°1321000
Add n°

Add n°1837000
Add n°1838000
Add n°1839000
Add n°1840000
Add n°1841000
Add n°1842000
Add n°1843000
Add n°1844000
Add n°1845000
Add n°1846000
Add n°1847000
Add n°1848000
Add n°1849000
Add n°1850000
Add n°1851000
Add n°1852000
Add n°1853000
Add n°1854000
Add n°1855000
Add n°1856000
Add n°1857000
Add n°1858000
Add n°1859000
Add n°1860000
Add n°1861000
Add n°1862000
Add n°1863000
Add n°1864000
Add n°1865000
Add n°1866000
Add n°1867000
Add n°1868000
Add n°1869000
Add n°1870000
Add n°1871000
Add n°1872000
Add n°1873000
Add n°1874000
Add n°1875000
Add n°1876000
Add n°1877000
Add n°1878000
Add n°1879000
Add n°1880000
Add n°1881000
Add n°1882000
Add n°1883000
Add n°1884000
Add n°1885000
Add n°1886000
Add n°1887000
Add n°1888000
Add n°1889000
Add n°1890000
Add n°1891000
Add n°1892000
Add n°1893000
Add n°1894000
Add n°1895000
Add n°1896000
Add n°1897000
Add n°1898000
Add n°1899000
Add n°1900000
Add n°1901000
Add n°1902000
Add n°1903000
Add n°1904000
Add n°1905000
Add n°1906000
Add n°1907000
Add n°

Add n°2423000
Add n°2424000
Add n°2425000
Add n°2426000
Add n°2427000
Add n°2428000
Add n°2429000
Add n°2430000
Add n°2431000
Add n°2432000
Add n°2433000
Add n°2434000
Add n°2435000
Add n°2436000
Add n°2437000
Add n°2438000
Add n°2439000
Add n°2440000
Add n°2441000
Add n°2442000
Add n°2443000
Add n°2444000
Add n°2445000
Add n°2446000
Add n°2447000
Add n°2448000
Add n°2449000
Add n°2450000
Add n°2451000
Add n°2452000
Add n°2453000
Add n°2454000
Add n°2455000
Add n°2456000
Add n°2457000
Add n°2458000
Add n°2459000
Add n°2460000
Add n°2461000
Add n°2462000
Add n°2463000
Add n°2464000
Add n°2465000
Add n°2466000
Add n°2467000
Add n°2468000
Add n°2469000
Add n°2470000
Add n°2471000
Add n°2472000
Add n°2473000
Add n°2474000
Add n°2475000
Add n°2476000
Add n°2477000
Add n°2478000
Add n°2479000
Add n°2480000
Add n°2481000
Add n°2482000
Add n°2483000
Add n°2484000
Add n°2485000
Add n°2486000
Add n°2487000
Add n°2488000
Add n°2489000
Add n°2490000
Add n°2491000
Add n°2492000
Add n°2493000
Add n°

Add n°3009000
Add n°3010000
Add n°3011000
Add n°3012000
Add n°3013000
Add n°3014000
Add n°3015000
Add n°3016000
Add n°3017000
Add n°3018000
Add n°3019000
Add n°3020000
Add n°3021000
Add n°3022000
Add n°3023000
Add n°3024000
Add n°3025000
Add n°3026000
Add n°3027000
Add n°3028000
Add n°3029000
Add n°3030000
Add n°3031000
Add n°3032000
Add n°3033000
Add n°3034000
Add n°3035000
Add n°3036000
Add n°3037000
Add n°3038000
Add n°3039000
Add n°3040000
Add n°3041000
Add n°3042000
Add n°3043000
Add n°3044000
Add n°3045000
Add n°3046000
Add n°3047000
Add n°3048000
Add n°3049000
Add n°3050000
Add n°3051000
Add n°3052000
Add n°3053000
Add n°3054000
Add n°3055000
Add n°3056000
Add n°3057000
Add n°3058000
Add n°3059000
Add n°3060000
Add n°3061000
Add n°3062000
Add n°3063000
Add n°3064000
Add n°3065000
Add n°3066000
Add n°3067000
Add n°3068000
Add n°3069000
Add n°3070000
Add n°3071000
Add n°3072000
Add n°3073000
Add n°3074000
Add n°3075000
Add n°3076000
Add n°3077000
Add n°3078000
Add n°3079000
Add n°

Add n°3595000
Add n°3596000
Add n°3597000
Add n°3598000
Add n°3599000
Add n°3600000
Add n°3601000
Add n°3602000
Add n°3603000
Add n°3604000
Add n°3605000
Add n°3606000
Add n°3607000
Add n°3608000
Add n°3609000
Add n°3610000
Add n°3611000
Add n°3612000
Add n°3613000
Add n°3614000
Add n°3615000
Add n°3616000
Add n°3617000
Add n°3618000
Add n°3619000
Add n°3620000
Add n°3621000
Add n°3622000
Add n°3623000
Add n°3624000
Add n°3625000
Add n°3626000
Add n°3627000
Add n°3628000
Add n°3629000
Add n°3630000
Add n°3631000
Add n°3632000
Add n°3633000
Add n°3634000
Add n°3635000
Add n°3636000
Add n°3637000
Add n°3638000
Add n°3639000
Add n°3640000
Add n°3641000
Add n°3642000
Add n°3643000
Add n°3644000
Add n°3645000
Add n°3646000
Add n°3647000
Add n°3648000
Add n°3649000
Add n°3650000
Add n°3651000
Add n°3652000
Add n°3653000
Add n°3654000
Add n°3655000
Add n°3656000
Add n°3657000
Add n°3658000
Add n°3659000
Add n°3660000
Add n°3661000
Add n°3662000
Add n°3663000
Add n°3664000
Add n°3665000
Add n°